In [1]:
import sys
import time
from charm.toolbox.eccurve import secp256k1, secp192k1, secp160k1
from blindIssuing_ecc import initialization, tracer_choose_keypair, Issuer, User, verify
import pandas as pd

In [2]:
L = secp192k1
round = 10
m = b'my msg'

### Issuing Test

In [3]:
def issue_test(L, round, time_recoder=[], flag=1):
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    total = 0
    for x in range(round):
        start = time.time()
        issuer = Issuer(params,tkey)
        # get the hash 
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)

        end = time.time()
        runtime = end - start
        time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag :print(msg.format(func = '[Credential Issuing]', time = runtime))
    if flag :print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [4]:
issue_test(L, round)

[Credential Issuing] takes 0.004769086837768555 second to complete
[Credential Issuing] takes 0.0046539306640625 second to complete
[Credential Issuing] takes 0.004614591598510742 second to complete
[Credential Issuing] takes 0.004714012145996094 second to complete
[Credential Issuing] takes 0.004824399948120117 second to complete
[Credential Issuing] takes 0.004595279693603516 second to complete
[Credential Issuing] takes 0.0046825408935546875 second to complete
[Credential Issuing] takes 0.004731416702270508 second to complete
[Credential Issuing] takes 0.004714488983154297 second to complete
[Credential Issuing] takes 0.004746198654174805 second to complete
The average time is 0.004704594612121582 second for 10 tests


---

### Parameter Test

In [5]:
def parameter_test(L, round, time_recoder=[], flag=1):
    total = 0
    for x in range(round):
        start = time.time()
        params = initialization(L)
        traceKeypair = tracer_choose_keypair(params)
        tkey = traceKeypair.yt
        xt = traceKeypair.xt
        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Parameter Generation]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [6]:
parameter_test(L, round)

[Parameter Generation] takes 0.0006766319274902344 second to complete
[Parameter Generation] takes 0.0005941390991210938 second to complete
[Parameter Generation] takes 0.0005698204040527344 second to complete
[Parameter Generation] takes 0.0005195140838623047 second to complete
[Parameter Generation] takes 0.0005152225494384766 second to complete
[Parameter Generation] takes 0.0005269050598144531 second to complete
[Parameter Generation] takes 0.0005404949188232422 second to complete
[Parameter Generation] takes 0.0005435943603515625 second to complete
[Parameter Generation] takes 0.0005245208740234375 second to complete
[Parameter Generation] takes 0.0005371570587158203 second to complete
The average time is 0.0005548000335693359 second for 10 tests


---

### Verifying Test

In [7]:
def verifying_test(L, round, time_recoder=[], flag=1):
    total = 0
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    
    for x in range(round):
        issuer = Issuer(params, tkey)
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()    
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)
        
        start = time.time()
        verify(roi, pi, sigma1, sigma2, delta, params, m, user.y, user.zeta1,user.zeta2, user.z)
        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Credential Verifying]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [8]:
verifying_test(L, round)

[Credential Verifying] takes 0.0014700889587402344 second to complete
[Credential Verifying] takes 0.0015115737915039062 second to complete
[Credential Verifying] takes 0.0015730857849121094 second to complete
[Credential Verifying] takes 0.0015149116516113281 second to complete
[Credential Verifying] takes 0.001596689224243164 second to complete
[Credential Verifying] takes 0.0015096664428710938 second to complete
[Credential Verifying] takes 0.0017621517181396484 second to complete
[Credential Verifying] takes 0.0015745162963867188 second to complete
[Credential Verifying] takes 0.0015840530395507812 second to complete
[Credential Verifying] takes 0.0016064643859863281 second to complete
The average time is 0.0015703201293945312 second for 10 tests


---

In [9]:
times = 1000
issue_time = []
parameter_time = []
verify_time = []
identify = ['Average Time']
identify.extend(['#'+str(x) for x in range(1,times + 1)])
issue_test(L, times, issue_time, 0)
parameter_test(L, times, parameter_time, 0)
verifying_test(L, times, verify_time, 0)
avg1 = sum(issue_time)/times
avg2 = sum(parameter_time)/times
avg3 = sum(verify_time)/times
issue_time.insert(0, avg1)
parameter_time.insert(0, avg2)
verify_time.insert(0, avg3)
dataframe = pd.DataFrame({'Rounds [TEST DATE: ' + time.strftime('%Y-%m-%d %H:%M',time.localtime(time.time())) + ']':identify, 'Issuing Time':issue_time, 'Parameter Time': parameter_time, 'Verifying Time': verify_time})
dataframe.to_csv('./ECC_192_result.csv', index=False)